<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 22</h1>

<b>Difference-in-differences (DID)</b> estimation is an extremely useful tool that can take several forms. Let us finish up our exploration of the second generalization of DID, which we call <b>panel fixed effects</b>. This is an estimation strategy when there are many more groups, which can be treated and then untreated, as we will see. This is the topic of section 5.2 in <i>Mastering Metrics</i>.

Learning objectives:

1. Examine panel data on the Minimum Legal Drinking Age (MLDA)
2. Run a DID using panel fixed effects and a clever TREAT $\times$ POST in `lm()`
3. Fail and then succeed. Failure is part of the story!
4. plot( ) time series on the same axes
5. Include other potential confounders like state-specific trends and beer taxes

In [ ]:
library(haven)
library(dplyr)
install.packages("estimatr")
library(estimatr)
library("foreign")
library(ggplot2)

Now let us load in the dataset `deaths.dta` that Angrist and Pischke examine in Table 5.2 and 5.3 inside section 5.2.

In [ ]:
deaths <- read_dta("data/deaths.dta")
head(deaths)

This panel dataset is complicated. Each row is actually not a state-year, it is instead a <b>state-year-age_group-cause</b>. That is, variable `dtype` indexes the cause of death for which that row's `mrate` measures the mortality rate in deaths per 100,000; deaths and population at risk are for the age group indexed by variable `agegr`.

The codings for each of these variables are:

`dtype`
1. All deaths
2. Motor vehicle accidents
3. Suicide
4. Homicide
5. Other external cause
6. Internal cause 

`agegr`
1. ages 15-17
2. ages 18-20
3. ages 21-24



This kind of dataset is challenging to work with, although it is perfectly set up for panel analysis. Simply put, it is hard to see the trees for the forest, to turn a common idiom on its head. It is not easy to peer inside these complex data.

Inside __R__, I think the most obvious way to peer inside the data is to use `subset()` to carve off a piece of the whole. 

In [ ]:
# This is data for California showing a bunch of variables between 1970 and 1996, 
# and the outcome variable mrate = Motor Vehicle Accident mortality per 100,000 aged 18-20 
ca_18_20_mva <- subset(deaths, state == 6 & agegr == 2 & dtype == 2)
ca_18_20_mva

There is not a lot going on here with the minimum legal drinking age (MLDA), because California had it set at 21 since 1933, apparently. [Source: Wikipedia](https://en.wikipedia.org/wiki/U.S._history_of_alcohol_minimum_purchase_age_by_state).

A pair of states that are more interesting to look at, per Angrist and Pischke, are Alabama and Arkansas. Alabama lowered its minimum legal drinking age to 19 in 1975, while Arkansas remained at 21 since Prohibition, like California.

In [ ]:
# Alabama is state FIPS == 1
al_18_20_mva <- subset(deaths, state == 1 & agegr == 2 & dtype == 2 & year <= 1983)
al_18_20_mva
# Arkansas is state FIPS == 5
ar_18_20_mva <- subset(deaths, state == 5 & agegr == 2 & dtype == 2 & year <= 1983)
ar_18_20_mva

To run a panel fixed effects model, we need to stack or pool these two sets of state-specific observations. In __R__, `rbind()` allows us to do exactly that.

In [ ]:
ar_al_18_20_mva <- rbind(al_18_20_mva, ar_18_20_mva)
ar_al_18_20_mva

Now let's call `mutate()` several times to create new variables for our standard DID set:

* $TREAT$ is an indicator of the treatment state, here Alabama
* $POST$ is an indicator of being past the treatment, here 1975 and onward
* $TREAT \times POST$ is the DID treatment variable, which measures being the treatment group during the treatment

In [ ]:
# TREAT == 1 when state == 1, for Alabama
ar_al_18_20_mva <- mutate(ar_al_18_20_mva, 
                          treat = as.integer(state == 1))

# POST == 1 when year >= 1975, which is when Alabama started to
# lower its minimum legal drinking age
ar_al_18_20_mva <- mutate(ar_al_18_20_mva, 
                          post = as.integer(year >= 1975))

# TREAT x POST is the product. It registers 1 when the 
# treated state (Alabama) is actually treated
ar_al_18_20_mva <- mutate(ar_al_18_20_mva, 
                          treatxpost = as.integer(treat*post))

In [ ]:
ar_al_18_20_mva

Now let us run regression DID with Alabama and Arkansas from 1970-1983. Here is the regression equation, which is also shown on page 192 of <i>Mastering Metrics</i>:

$$
mrate_{st} = \alpha + \beta \ TREAT_s + \gamma \ POST_t + \delta_{rDID} \ TREAT_s \times POST_t + e_{st} 
$$

where $mrate_{st}$ is the motor vehicle accident mortality rate in state $s$ at time $t$. (Angrist and Pischke are silent on which mortality rate they are talking about. Total or MVA mortality are the two obvious choices.) 

In [ ]:
ar_al_did_70_83 <- lm(mrate ~ treat + post + treatxpost,
               data = ar_al_18_20_mva)
summary(ar_al_did_70_83)

What do we see in the results above? Are you impressed or disheartened? Are Alabama and Arkansas good states to compare? Is the time period very long?

<hr>

Another thing we could do is try and visualize what is happening. It turns out that the death rates are pretty easy to superimpose on the same axes, even with my poor graphing skills in __R__!

In [ ]:
# These are the "pre-stacked" data frames
plot(ar_18_20_mva$year, ar_18_20_mva$mrate, col = "red", type = "b", 
     main = "Arkansas (red) and Alabama (blue) MVA death rates")
lines(al_18_20_mva$year, al_18_20_mva$mrate, col = "blue")
grid()

There is some evidence of similar trending, I suppose, but it is not exactly breathtaking, is it! 

Below is the clever Angrist-Pischke measure of treatment, $LEGAL_{st}$, which is a little different from $TREAT_s \times POST_t$ because it measures the share of 18-20 year olds who can legally drink. That makes a lot of sense given that the left-hand-side variable is the death rate among ages 18-20y. The variable $LEGAL_{st}$ tends to be near either 0, 1/3, 2/3, or 1 for obvious reasons.

In [ ]:
plot(al_18_20_mva$year, al_18_20_mva$legal, col = "blue", type = "b")
lines(ar_18_20_mva$year, ar_18_20_mva$legal, col = "red")

Perhaps to no great surprise, Arkansas does not appear to be a particularly good control group for Alabama. Visually speaking, parallel trends are far from easy to pick out here.

Takeaways:
1. It is OK to be "wrong." Everybody is wrong often. Breathe it in and feel the wrong set you free
2. When data are annual averages, it can be easy not to find statistically significant results

Before we leave the Alabama and Arkansas example, let us examine the clever trick introduced by Angrist and Pischke in <i>Mastering Metrics</i> section 5.2, where they propose a new DID treatment variable for the minimum legal drinking age scenario, called $LEGAL_{st}$, which is an index ranging between 0 and 1 for state $s$ in year $t$.

Because states changed their MLDAs between 18 and 21, the age group 18-20 is most interesting to examine. But it also is not equally interesting in all places, because different states did different things. They also did them at <i>different times</i> during the year, passing a law that affected less than the full year, for example.

(An additional wrinkle that MM can largely sidestep is that alcohol-related motor vehicle accidents are surely not uniformly distributed across the days and weeks of the year. With annual data, these issues are unlikely to be important. But if the data were quarterly, monthly, weekly, or daily, these issues would be important. For more, see the ECON 140R modules on time series and seasonal adjustment.) 

Bottom line, $LEGAL_{st}$ is a constructed variable that helps estimation while still looking a lot like the $TREAT_s \times POST_t$ variable that we know and love. Let us examine what happens in the case of Alabama and Arkansas when we use $LEGAL_{st}$ instead:

In [ ]:
ar_al_did_70_83_2 <- lm(mrate ~ treat + post + legal,
               data = ar_al_18_20_mva)
summary(ar_al_did_70_83_2)

What do we see here? Is the story much different using $LEGAL_{st}$?

<hr>

Let us jump ahead to the finish line in Chapter 5, which is Table 5.2, which shows the results of four different models (across the columns) applied to four different measures of mortality (across the rows). Here is Table 5.2:

<img src = "images/MMtbl52.png" width = 500 /> 

The way to read the table is that each cell is the DID estimate of the effect of a state's <i>lowering</i> its minimum legal drinking age (MLDA) on a particular death rate, which is shown along the row. A positive number is an increase in the death rate that is caused by lowering the MLDA, so that more 18-20 year olds can legally drink alcohol. The number in the upper left, $10.80$, means that expanding legal drinking down to age 18 triggers an increase in the total death rate among 18-20 year olds of 10.80 deaths per 100,000. That estimate is obtained without state trends or weights; when those are included, along the columns, the results change a little.

See that $10.80$? Here it is below, in the first row after the intercept. We are just running `lm()` with the all-cause death rate as our $y$-variable (`dtype1` == 1), in the appropriate time range and with the 18-20yo age group.

In [ ]:
panel_fe_1 <- lm(mrate ~ legal + factor(state) + factor(year),
                 data = subset(deaths, year <= 1983 & agegr == 2 & dtype == 1))
summary(panel_fe_1)

Note how the standard error of that coefficient on $LEGAL_{st}$ is $3.14$ rather than the reported value of $4.59$ shown in Table 5.2. As Angrist and Pischke discuss in the appendix to Chapter 5, best practices are to <i>cluster the standard errors</i> on the group ID, here the states.

In the long, long ago we did the same for the RAND HIE, only there we clustered on `famid`, the family ID. So let us bring back our beloved friend `lm_robust()` for some good times:

In [ ]:
panel_fe_1r <- lm_robust(mrate ~ legal + factor(state) + factor(year),
                         data = subset(deaths, year <= 1983 & agegr == 2 & dtype == 1),
                         clusters = state, se_type = "stata")
summary(panel_fe_1r)

Ha! Note the call to ask __R__ to use `se_type` equal to "stata" there as a finishing touch!

<hr>

<h2>Interlude: Unit-specific time trends</h2> 

In *Probing DID Assumptions* inside Section 5.2, Angrist and Pischke introduce state-specific linear time trends, to check the robustness of the MLDA result. These are the "State trends" listed in the second-to-last row in Table 5.2 and on the right-hand side of Table 5.3. Operationally, we mean variables that shadow `year` and only "switch on" for an individual state.

One might want to model this if it seems appropriate. If trends are not strictly parallel but appear to be following some kind of "business as usual," a DID approach can still reveal a causal effect if we model the trends correctly. If reality looks like FIgure 5.6, for example, then unit-specific time trends might be the right modeling approach

<img src = "images/MMfig56.png" width = 500 /> 

It might help to visualize what we mean in the small Fed District dataset from Section 5.1.

In [ ]:
figure_3_annavg <- read_dta("data/figure_3_annavg.dta")
figure_3_annavg

When there are only 2 units, we can manually create interaction variables that will help us model unit-specific linear time trends in the regression. When there are many units, this becomes much more tedious!

In the code below, we generate new variables `d6y` and `d8y`, which get the year variable when the district variable is equal to 6 or 8 respectively, and get zero otherwise.

In [ ]:
figure_3_annavg$d6y = (figure_3_annavg$district == 6)*figure_3_annavg$year

figure_3_annavg$d8y = (figure_3_annavg$district == 8)*figure_3_annavg$year

figure_3_annavg

Here are the original results of the DID model run on these data, with $treat_d$, $post_t$, and $treat_d \times post_t$:

In [ ]:
banks_did <- lm(banks ~ treat 
                + post 
                + treatxpost,
               data = figure_3_annavg)
summary(banks_did)

And here are the more generalized results, when we also include district-specific linear time trends: 

In [ ]:
banks_did_tt <- lm(banks ~ treat
                   + post 
                   + treatxpost
                   + d6y + d8y,
                   data = figure_3_annavg)
summary(banks_did_tt)

Here is a shortcut to the same results, using `factor(district)*year` within the equation inside the call to `lm()`. Note how the format of the time trends is a little different. By default, __R__ tosses in a common linear `year` effect, which is the same as the `d6y` effect above. Then it also estimates the differential time effect associated with district 8, and the sum of those two below is the `d8y` that we saw above.

In [ ]:
banks_did_tt2 <- lm(banks ~ treat
                    + post 
                    + treatxpost
                    + factor(district)*year,
                    data = figure_3_annavg)
summary(banks_did_tt2)

<hr>

<h2>More control variables in panel fixed effects
</h2>
    
The last topic in Chapter 5 is introducing another control variable to a panel fixed effects regression model. This turns out to be as easy as inserting the confounder of interest as another right-hand-side variable.

<img src = "images/MMtbl53.png" width = 500 /> 

The code below replicates the estimates in columns (1) and (2) in the first row in Table 5.3 above. Both those estimates are from a single regression of All deaths (`mrate`) on Fraction legal (`legal`), Beer tax (`beertaxa`), and a big set of indicator controls for state and for year. Note the large standard error on the beer tax coefficient.

In [ ]:
panel_fe_1r <- lm_robust(mrate ~ legal + 
                         beertaxa + 
                         factor(state) + 
                         factor(year),
                         data = subset(deaths, year <= 1983 & agegr == 2 & dtype == 1),
                         clusters = state, se_type = "stata")
summary(panel_fe_1r)

What do you see here? What is the bottom line for beer taxes and minimum legal drinking ages (MLDA) in terms of their effects on the death rate for people aged 18-20y?

<hr>

And below are the results for columns (3) and (4) in the first row in Table 5.3 above, in which we are also modeling state-specific linear time trends in All deaths (`mrate`). The part of the code that models state-specific linear time trends is `factor(state)*year` on the right-hand side of the equation inside the call to `lm()`.

Note the VERY large standard error on the beer tax coefficient.

In [ ]:
options(scipen = 999)

In [ ]:
panel_fe_1rr <- lm_robust(mrate ~ legal + beertaxa + factor(state) + factor(year)
                         + factor(state)*year,
                         data = subset(deaths, year <= 1983 & agegr == 2 & dtype == 1),
                         clusters = state, se_type = "stata")
summary(panel_fe_1rr)

<hr>

<h2>If there's time: Another successful DID failure(?)</h2>

Courtesy of Prof. Ethan Lewis at Dartmouth, we have another interesting DID regression or two up our sleeves. Recall the famous Card and Krueger paper about the rise in the minimum wage in New Jersey in 1992 compared to Pennsylvania? Then, it rose from \\$4.25 to \\$5.05 per hour.

<b>In 2014</b>, New Jersey raised its minimum wage from \\$7.25 to \\$8.25, and then a little more thereafter, to keep up with inflation. Like before, Pennsylvania left its minimum wage alone. Both times series can be found on the St. Louis Fed FRED website: [for NJ](https://fred.stlouisfed.org/series/STTMINWGNJ) and [for PA](https://fred.stlouisfed.org/series/STTMINWGPA). With a little extra work, one can quickly edit one of those two graphs to show both series together.

<h3>A note on timing</h3> 

According to the text of the [New Jersey Administrative Code 12:56-3.1](https://www.nj.gov/labor/forms_pdfs/lwdhome/MinWage2015.pdf), this change was effective on January 1, 2014. Labor surveys like the Current Population Survey (CPS) and American Community Survey (ACS) measure employment at a point in time, usually "last week," while earnings are usually either reported for the previous year (ACS) or a different time period (CPS). A measure of employment in New Jersey in the 2014 wave of ACS indicates the average level of employment during the year, and thus it should fully reflect conditions under the increased minimum wage.

This might be fun to examine. A big problem, as we've discussed in class, is that for a DID to be successful, you need to measure the units more likely to show an effect from treatment. Card and Krueger actually surveyed fast food restaurants, who employ lots of minimum-wage workers.

What we can do is look at annual averages of employment in the Census Bureau's American Community Survey (ACS) courtesy of IPUMS. We have two takes on this below: one that measures all employment in NJ and PA, and another that measures employment of just men with high school or less and aged 16-24. 

<h3>Creating the extract</h3> 

There are several methods and data sources one could use to create a panel dataset with employment outcomes in NJ and PA around the time of the minimum wage law change.

1. Download microdata from [usa.ipums.org](https://usa.ipums.org) and collapse the data into averages by year and state
2. Use the "Analyze Data Online" tool at [usa.ipums.org](https://usa.ipums.org) to create state-level averages
3. See whether FRED or another handy database might contain the same or similar data

FRED appears to contain [monthly employment statistics for New Jersey](https://fred.stlouisfed.org/series/LASST340000000000005) from the Bureau of Labor Statistics; in the small print at the bottom of that page, FRED explains that the data are from the monthly CPS. I don't think FRED contains series for employment by education level, but that would be possible to measure inside IPUMS.

To create the extracts below, I chose option 1 above. But FRED might be a good option too. Below is a graph I produced using FRED:

<img src = "images/nj-pa-emp-2010-2023.png" width = 1000 /> 

In [ ]:
# These data are state-year averages drawn from usa.ipums.org 
# All employment
njpa1117_annavgemp <- read_dta("data/njpa1117_annavgemp.dta")
# Employment of just men with HS or less, aged 16-24. I misnamed this with a "25"
njpa1117_m1625_hs_annavgemp <- read_dta("data/njpa1117_m1625_hs_annavgemp.dta")

# Let's look at one of these stacked datasets
njpa1117_annavgemp

Let's look at the data. This feels clumsy to me in __R__, because even `ggplot` is not very good at graphing time series on the same axes. With a short panel, we could just dump the data into a spreadsheet instead. But instead, let's struggle through. Deep breath.

Here is code that cuts and pastes the columns into a new data frame with a single column for year, and then columns for NJ employment and PA employment:

In [ ]:
# Create a data frame with named columns
njpa <- data.frame(
  year = subset(njpa1117_annavgemp, statefip == 34)$year,
  nj = subset(njpa1117_annavgemp, statefip == 34)$sum_employed,
  pa = subset(njpa1117_annavgemp, statefip == 42)$sum_employed
)

njpa

In [ ]:
# Same, but with men 16-24 with high school or less
njpa_m1625_hs <- data.frame(
  year = subset(njpa1117_m1625_hs_annavgemp, statefip == 34)$year,
  nj = subset(njpa1117_m1625_hs_annavgemp, statefip == 34)$sum_employed,
  pa = subset(njpa1117_m1625_hs_annavgemp, statefip == 42)$sum_employed
)

njpa_m1625_hs

Here is a visualization. The challenge is that employment is considerably larger in PA compared to NJ, and thus showing them on the same axes apparently requires a scaling trick in __R__ as of this writing.

<b>I owe a huge thanks to ChatGPT for helping with this</b>. But I would also say that ChatGPT's assistance was not exactly off-the-shelf perfect. Googling and finding StackExchange discussions also was not super helpful either.

I coded the following by setting `scalar0` to a number between 0 and 1 through trial and error. The scaling is set to appropriately position the PA series, which is on the secondary $Y$-axis.

In [ ]:
scalar0 = 0.725

ggplot(njpa, aes(x = year)) +
  geom_line(aes(y = nj, color = "NJ"), size = 1) +
  geom_line(aes(y = pa*scalar0, color = "PA"), linewidth = 1, linetype = "dashed") +
  scale_y_continuous(name = "NJ axis", 
                     sec.axis = sec_axis(~.*(1/scalar0), name = "PA axis")
                     )+
  labs(title = "Total employment in NJ and PA") +
  theme_minimal() +
  theme(axis.title.y = element_text(color = "blue"), 
        axis.title.y.right = element_text(color = "red")
        )

And here is the same thing, but with a different scaling, for employment of men 16-24y with high school or less:

In [ ]:
scalar0 = 0.625

ggplot(njpa_m1625_hs, aes(x = year)) +
  geom_line(aes(y = nj, color = "NJ"), size = 1) +
  geom_line(aes(y = pa*scalar0, color = "PA"), linewidth = 1, linetype = "dashed") +
  scale_y_continuous(name = "NJ axis", 
                     sec.axis = sec_axis(~.*(1/scalar0), name = "PA axis")
                     )+
  labs(title = "Employment of men 16-24 with ≤ HS in NJ and PA") +
  theme_minimal() +
  theme(axis.title.y = element_text(color = "blue"), 
        axis.title.y.right = element_text(color = "red")
        )

The code below creates the familiar variables $TREAT_s$, $POST_t$, and $TREAT_s \times POST_t$ for use in a DID regression. First, let's look at total employment:

In [ ]:
njpa1117_annavgemp <- mutate(njpa1117_annavgemp, 
                             treat = as.integer(nj == 1))

njpa1117_annavgemp <- mutate(njpa1117_annavgemp, 
                             post = as.integer(year >= 2014))

njpa1117_annavgemp <- mutate(njpa1117_annavgemp, 
                             treatxpost = as.integer(treat*post))

njpa1117_annavgemp

And now here is our DID regression for <b>total employment</b>, using the `njpa1117_annavgemp` data:

$$
employment_{st} = \alpha + \beta \ TREAT_s + \gamma \ POST_t + \delta_{rDID} \ TREAT_s \times POST_t + e_{st} 
$$


In [ ]:
njpa1117_reg1 <- lm(sum_employed ~ treat + post + treatxpost,
                   data = njpa1117_annavgemp)
summary(njpa1117_reg1)

Hmm. What do you see in this DID regression of total employment on the 2014 minimum wage increase? Are you convinced of anything?

<hr>

Now let us also look at men aged 16-25 with a high school degree or less. Here are the DID variables:

In [ ]:
njpa1117_m1625_hs_annavgemp <- mutate(njpa1117_m1625_hs_annavgemp,
                                      treat = as.integer(nj == 1))

njpa1117_m1625_hs_annavgemp <- mutate(njpa1117_m1625_hs_annavgemp,
                                      post = as.integer(year >= 2014))

njpa1117_m1625_hs_annavgemp <- mutate(njpa1117_m1625_hs_annavgemp,
                                      treatxpost = as.integer(treat*post))

njpa1117_m1625_hs_annavgemp

And now here is our DID regression for <b>employment of men aged 16-24 with high school or less</b>, using the `njpa1117_m1625_hs_annavgemp` data:

$$
employment^{m16-24hs}_{st} = \alpha + \beta \ TREAT_s + \gamma \ POST_t + \delta_{rDID} \ TREAT_s \times POST_t + e_{st} 
$$


In [ ]:
njpa1117_reg2 <- lm(sum_employed ~ treat + post + treatxpost,
                   data = njpa1117_m1625_hs_annavgemp)
summary(njpa1117_reg2)

Hmm. What do you see in this DID regression of total employment on the 2014 minimum wage increase? Are you convinced of anything?

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>